In [1]:
# coding: utf-8
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing, model_selection

In [2]:
def ginic(actual, pred):
    actual = np.asarray(actual) #In case, someone passes Series or list
    n = 50
    a_s = actual[np.argsort(pred)]
    a_c = a_s.cumsum()
    giniSum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
    return giniSum / n

# ## Load Data
train = pd.read_csv("data/train.csv")
#test = pd.read_csv("data/test.csv")

d_median = train.median(axis=0)
d_mean = train.mean(axis=0)

def transform_df(df):
    df = pd.DataFrame(df)
    dcol = [c for c in df.columns if c not in ['id','target']]
    df['ps_car_13_x_ps_reg_03'] = df['ps_car_13'] * df['ps_reg_03']
    df['negative_one_vals'] = np.sum((df[dcol]==-1).values, axis=1)
    for c in dcol:
        if '_bin' not in c:
            df[c+str('_median_range')] = (df[c].values > d_median[c]).astype(int)
            df[c+str('_mean_range')] = (df[c].values > d_mean[c]).astype(int)
    return df

def prepare_data(dataset,Useless_fatures, name_target = 'target', replace_Nan = 0):
    if replace_Nan != False:
        dataset.replace(-1, replace_Nan )

    if name_target in dataset:
        X = dataset.drop([name_target],1)
        X = X.drop(Useless_fatures,1)
        
        label = pd.DataFrame()
        label['real'] = dataset[name_target]
        label['inv']  = 1-dataset[name_target]
        y = label.as_matrix()
        print(len(X), " ",len(label))
        return X , y
    else:
        return dataset.drop(Useless_fatures,1)
    


In [3]:
def make_sub(test,model,sess,best_validation_accuracy = ''):
    print('creating submission results...')
    saver.restore(session,'models/my_model')
    #X_sub = transform_df(test)
    X_sub = prepare_data(test,['id'])
    sub1 = sess.run(model,feed_dict={x:X_sub[:int(892816/2)]})
    sub2 = sess.run(model,feed_dict={x:X_sub[int(892816/2):]})
    sub_tot = np.append(sub1,sub2,axis=0)
    max_index= sub_tot.argmax(1)
    max_res = np.amax(sub_tot,1)
    sub_tot = [n if max_index[counter] == 1 else 1-n for counter,n in zip(range(len(max_res)),max_res)]
    print(len(sub_tot))
    test_id = test.id.values    
    print(len(test_id))
    sub = pd.DataFrame()
    sub['id'] = test_id
    sub['target'] = sub_tot
    sub.to_csv('submission_{}.csv'.format(int(best_validation_accuracy*100)) ,index = False)
    print('Done!')

In [24]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
def print_accuracy( r_valu = False, feed = feed_dict_test):
    # Use TensorFlow to compute the accuracy.
    acc = session.run(accuracy, feed_dict=feed)
    if r_valu:
        return acc
    # Print the accuracy.
    print("Accuracy on test-set: {0:.1%}".format(acc))

In [9]:
target_0 = train['target']==0
target_1 = train['target']==1
train_down_samp = train[target_1].append(train[target_0][0:int(sum(target_1)*3)])
train_down_samp = train_down_samp.append(train[target_1])
train_down_samp = train_down_samp.append(train[target_1])
#sum(new_train['target']==1)/len(new_train)

Useless_fatures = ['ps_ind_10_bin','ps_ind_11_bin','ps_ind_13_bin','ps_calc_20_bin','ps_ind_12_bin','ps_calc_15_bin','id']
#Useless_fatures = train.columns[train.columns.str.startswith('ps_calc_')] 
#new_df_train = transform_df(train_down_samp)
X_50 , y_50 = prepare_data(train_down_samp,['id'])
X_real_test, y_real_test = prepare_data(train[0:3000],['id'])
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_50 , y_50, test_size=0.02)

print("Size of:")
print("- Training-set:\t\t{}".format(len(X_train)))
print("- Test-set:\t\t{}".format(len(X_test)))


130164   130164
3000   3000
Size of:
- Training-set:		127560
- Test-set:		2604


In [20]:
#input size
input_size = X_train.shape[1]
num_classes = 2 #y_test.shape[1]


# ### Placeholder variables
x = tf.placeholder(tf.float32, [None, input_size])
y_true = tf.placeholder(tf.float32, [None,num_classes]) ##[None, num_classes]
y_true_cls = tf.argmax(y_true, axis=1)

# ### Helper-functions for creating new variables
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))


# ### Helper-function for creating a new Fully-Connected Layer
def new_fc_layer(input,          # The previous layer.
                 num_inputs,     # Num. inputs from prev. layer.
                 num_outputs,    # Num. outputs.
                 use_relu=True,  # Use Rectified Linear Unit (ReLU)?
                 use_sig=False):
    # Create new weights and biases.
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    # Calculate the layer as the matrix multiplication of
    # the input and weights, and then add the bias-values.
    layer = tf.matmul(input, weights) + biases

    # Use ReLU?
    if use_relu:
        layer = tf.nn.relu(layer)
    elif use_sig:
        layer = tf.nn.sigmoid(layer)
    return layer


# ###  Layers
#hyperparameters
n_nodes_hl1 = 200
n_nodes_hl2 = 600
n_nodes_hl3 = 100

layer_h1 = new_fc_layer(input=x,
                         num_inputs=input_size,
                         num_outputs=n_nodes_hl1,
                         use_relu=True)

layer_h2 = new_fc_layer(input=layer_h1,
                         num_inputs=n_nodes_hl1,
                         num_outputs=n_nodes_hl2,
                         use_relu=True,
                         use_sig = False)

layer_h3 = new_fc_layer(input=layer_h2,
                         num_inputs=n_nodes_hl2,
                         num_outputs=n_nodes_hl3,
                         use_relu=False,
                         use_sig = True)

y_pred = new_fc_layer(input=layer_h3,
                         num_inputs=n_nodes_hl3,
                         num_outputs=num_classes,
                         use_relu=False,
                         use_sig = False)
y_pred_soft_max = tf.nn.softmax(y_pred)
y_pred_cls = tf.argmax(y_pred, axis=1)


# ### Cost-function and Optimizer
learning_rate = tf.placeholder(tf.float32, shape=[])
#global_step = tf.Variable(0, trainable=False)
#starter_learning_rate = 0.1
#learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           #100000, 0.96, staircase=True)


##########################
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=y_pred,
                                                        labels=y_true)
cost = tf.reduce_mean(cross_entropy)
#error = tf.square(y_pred - y_true)/2 
#cost = tf.reduce_sum(error + error* y_true*5)

optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate).minimize(cost)  #  1e-4
##########################

# ### Performance measures
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))




session = tf.Session()

session.run(tf.global_variables_initializer())


# ### Helper-functions to show performance
feed_dict_test = {x: X_test, y_true: y_test}  #.values.reshape([-1,1])

def print_accuracy( r_valu = False, feed = feed_dict_test):
    # Use TensorFlow to compute the accuracy.
    acc = session.run(accuracy, feed_dict=feed)
    if r_valu:
        return acc
    # Print the accuracy.
    print("Accuracy on test-set: {0:.1%}".format(acc))

print_accuracy()


saver = tf.train.Saver()  #search tf.saver.tensorflow

# ### Helper-function to perform optimization iterations
batch_size = 50
best_validation_accuracy = 0
def optimize( hm_epochs = 10, learning_r = 1e-4):
    global best_validation_accuracy
    global global_step
    for epoch in range(hm_epochs):
    		epoch_loss = 0
    		i = 0
    		len_train = X_train.shape[0]
    		while i < len_train:
    			start = i
    			end = i + batch_size
                #global_step++
                
    			batch_x = np.array(X_train[start:end])
    			batch_y = np.array(y_train[start:end])    #.values.reshape([-1,1]))
    			# Put the batch into a dict with the proper names
    			# for placeholder variables in the TensorFlow graph.
    			# Note that the placeholder for y_true_cls is not set
    			# because it is not used during training.

    			feed_dict_train = {x: batch_x,
    							   y_true: batch_y,
    							   learning_rate: learning_r}

    			# Run the optimizer using this batch of training data.
         		# TensorFlow assigns the variables in feed_dict_train
    			# to the placeholder variables and then runs the optimizer.
    			_, c = session.run([optimizer,cost], feed_dict=feed_dict_train)
    			#session.run(optimizer, feed_dict=feed_dict_train)
    			epoch_loss += c

    			i += batch_size

    			if i % 20000 == 0  or i == (len_train - 1):


    				acc_ = print_accuracy(True)
    				if acc_ > best_validation_accuracy:
                    # Update the best-known validation accuracy.
    					best_validation_accuracy = acc_
    					saver.save(session, 'models/my_model')
    					improved_str = '*'
    				else:
    					improved_str = ''

    				print('loss: {}, data used: {} / {},  acc: {}{}'.format(epoch_loss/20,i,len_train,acc_,improved_str))
    				epoch_loss = 0



        print('Epoch {} completed out of {}'.format(epoch+1,hm_epochs))

    print('Optimization ended '+ 10*'_' + 'best accuracy: {}'.format(best_validation_accuracy))


# ## optimization

Accuracy on test-set: 51.0%


In [33]:
best_validation_accuracy

0.76958525

In [42]:
optimize(30,1e-4)

loss: 2.693217629007995, data used: 20000 / 127560,  acc: 0.8076037168502808
loss: 2.755544687435031, data used: 40000 / 127560,  acc: 0.7991551756858826
loss: 2.732390321046114, data used: 60000 / 127560,  acc: 0.7972350716590881
loss: 2.699224737100303, data used: 80000 / 127560,  acc: 0.8018433451652527
loss: 2.7992658603936436, data used: 100000 / 127560,  acc: 0.7876343727111816
loss: 3.0008137419819834, data used: 120000 / 127560,  acc: 0.7918586730957031
loss: 2.8270096432417633, data used: 20000 / 127560,  acc: 0.8072196245193481
loss: 2.626630991511047, data used: 40000 / 127560,  acc: 0.79877108335495
loss: 2.65242955237627, data used: 60000 / 127560,  acc: 0.7991551160812378
loss: 2.6518537111580374, data used: 80000 / 127560,  acc: 0.8072196841239929
loss: 2.6340655889362097, data used: 100000 / 127560,  acc: 0.7845621705055237
loss: 2.7737218506634234, data used: 120000 / 127560,  acc: 0.7983871102333069
loss: 2.7876395028084517, data used: 20000 / 127560,  acc: 0.80606758

loss: 2.406407150719315, data used: 100000 / 127560,  acc: 0.7999231815338135
loss: 2.405604849755764, data used: 120000 / 127560,  acc: 0.8145160675048828
loss: 2.1946611884981393, data used: 20000 / 127560,  acc: 0.8149001598358154
loss: 2.0495502300560475, data used: 40000 / 127560,  acc: 0.8152841925621033
loss: 2.0950528444722294, data used: 60000 / 127560,  acc: 0.8160521984100342
loss: 2.132184525951743, data used: 80000 / 127560,  acc: 0.8179723620414734
loss: 2.351427699625492, data used: 100000 / 127560,  acc: 0.8018433451652527
loss: 2.4363565424457194, data used: 120000 / 127560,  acc: 0.8114439249038696
loss: 2.386872349306941, data used: 20000 / 127560,  acc: 0.8091397881507874
loss: 2.142755561042577, data used: 40000 / 127560,  acc: 0.8122119307518005
loss: 2.1847850672900675, data used: 60000 / 127560,  acc: 0.8087557554244995
loss: 2.25949579346925, data used: 80000 / 127560,  acc: 0.8125959634780884
loss: 2.217010240443051, data used: 100000 / 127560,  acc: 0.8079877

In [45]:
test = pd.read_csv("data/test.csv")

make_sub(test,y_pred_soft_max,session,best_validation_accuracy = best_validation_accuracy)

creating submission results...
INFO:tensorflow:Restoring parameters from models/my_model
892816
892816
Done!


In [43]:
feed_dict_real_test = {x: X_real_test, y_true:y_real_test}
print_accuracy(feed=feed_dict_real_test)
a = session.run(y_pred_soft_max,feed_dict_real_test)
b = a[:10]
c= b.argmax(1)
np.amax(b,1)
[n if c[m] == 1 else 1-n for m,n in zip(range(len(np.amax(b,1))),np.amax(b,1))]
#np.round(b,3)


Accuracy on test-set: 90.4%


[1.0,
 1.0,
 0.99982601,
 0.89293063,
 0.99038321,
 0.9428665,
 0.81507599,
 0.99999464,
 0.9560079,
 0.096562206745147705]

In [40]:
feed_dict_real_test[y_true][0:10]

array([[0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0]])

In [41]:
c

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [37]:
np.round(b,3)

array([[ 0.        ,  1.        ],
       [ 0.        ,  1.        ],
       [ 0.004     ,  0.99599999],
       [ 0.40700001,  0.59299999],
       [ 0.066     ,  0.93400002],
       [ 0.51300001,  0.48699999],
       [ 0.61299998,  0.38699999],
       [ 0.001     ,  0.99900001],
       [ 0.37099999,  0.62900001],
       [ 0.551     ,  0.449     ]], dtype=float32)